In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA,SparsePCA
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from scipy.stats import uniform, truncnorm, randint
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.utils import class_weight
import gensim
from gensim.models import KeyedVectors, Word2Vec, FastText
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix,classification_report

In [2]:
link = 'https://raw.githubusercontent.com/python-for-ml/sent-analysis/master/finaldataset.csv'
data = pd.read_csv(link)
data=data.drop(['title_x','title_y','title','value'],axis=1)
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,কিছুটা নেতিবাচক
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,কিছুটা নেতিবাচক
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,নিশ্চিত নেতিবাচক
3,চুরি নয় লুটপাট।,নিশ্চিত নেতিবাচক
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,নিরপেক্ষ


In [3]:
le = LabelEncoder()
data['tag'] = le.fit_transform(data['tag'])
data.head()

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,1
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,1
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,4
3,চুরি নয় লুটপাট।,4
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,2


In [4]:
data['tag']=data['tag'].replace(3, 0)
data['tag']=data['tag'].replace(4, 1)
data = data[data['tag'] != 2]

In [5]:
d = data['data'].tolist()
label = data['tag'].tolist()

In [6]:
for i in range(len(d)):
    d[i] = re.sub(r'[^\w\s]', '', d[i])

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
corpus = []
for i in range(len(d)):
    corpus.append(word_tokenize(d[i]))

In [9]:
vectorizer=CountVectorizer()
vocabulary=vectorizer.fit(d)
X= vectorizer.transform(d)
size = round(len(vocabulary.get_feature_names())**0.65)

In [10]:
size

664

In [11]:
wvmodel = Word2Vec(corpus, size=size, window=4, min_count=1)

In [12]:
X = np.zeros((len(corpus),size)) #Initializing the X matrix with zeros
for i in range(len(corpus)):
    emb = np.zeros((1,size))
    for w in corpus[i]:
        emb = emb +  wvmodel[w]
    X[i] = emb 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,label, test_size=0.2)

In [14]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

In [15]:
class_weights


array([1.47218453, 0.75715283])

In [23]:
cw = {0:1.6*class_weights[0],1:class_weights[1]}
cw_DT = {0:1.3*class_weights[0],1:class_weights[1]}

In [42]:
param_grid = { 
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [49,48,50,52],
    'criterion' : ['gini', 'entropy']
}

In [43]:
rfc=RandomForestClassifier(class_weight = cw)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid)
CV_rfc.fit(X_train, y_train)
y_pred = CV_rfc.predict(X_test)
CV_rfc.best_params_

{'criterion': 'gini', 'max_depth': 48, 'max_features': 'auto'}

In [44]:
CV_rfc.best_score_

0.6361751152073734

In [45]:
print("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
print("Precision :",metrics.precision_score(y_test,y_pred))
print("F1-Score  :",metrics.f1_score(y_test,y_pred))
print("Recall    :",metrics.recall_score(y_test,y_pred))

Accuracy  : 0.6457853523721787
Precision : 0.6786096256684492
F1-Score  : 0.767462957363169
Recall    : 0.8830897703549061


In [46]:
print(confusion_matrix(y_test,y_pred))

[[ 133  601]
 [ 168 1269]]


In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.44      0.18      0.26       734
           1       0.68      0.88      0.77      1437

    accuracy                           0.65      2171
   macro avg       0.56      0.53      0.51      2171
weighted avg       0.60      0.65      0.59      2171



In [17]:
params = {'max_leaf_nodes': list(range(1, 10)), 'min_samples_split': list(range(1, 10)),'criterion' :['gini', 'entropy'],'max_depth':list(range(1, 10))}
grid_search_cv = GridSearchCV(tree.DecisionTreeClassifier(class_weight = cw_DT), params)
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                              class_weight={0: 1.4721845318860245,
                                                            1: 0.7571528262386602},
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                   

In [18]:
y_pred=grid_search_cv.predict(X_test)

In [19]:
print("Accuracy  :",metrics.accuracy_score(y_test, y_pred))
print("Precision :",metrics.precision_score(y_test,y_pred))
print("F1-Score  :",metrics.f1_score(y_test,y_pred))
print("Recall    :",metrics.recall_score(y_test,y_pred))

Accuracy  : 0.6568401658222017
Precision : 0.6696288552012546
F1-Score  : 0.7747202902933172
Recall    : 0.9189383070301291


In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.19      0.28       777
           1       0.67      0.92      0.77      1394

    accuracy                           0.66      2171
   macro avg       0.62      0.55      0.53      2171
weighted avg       0.63      0.66      0.60      2171



In [21]:
confusion_matrix(y_test,y_pred)

array([[ 145,  632],
       [ 113, 1281]])

In [22]:
 grid_search_cv.best_params_

{'criterion': 'gini',
 'max_depth': 2,
 'max_leaf_nodes': 3,
 'min_samples_split': 2}